## Scenario 1: A single data scientist participating in an ML competition

MLflow setup:
* Tracking server: no
* Backend store: local filesystem
* Artifacts store: local filesystem

The experiments can be explored locally by launching the MLflow UI.

In [15]:
import mlflow
from mlflow.exceptions import MlflowException

# ----------------------------------------------------------------------
# 💡 CRITICAL FIX: Setting tracking URI to a local path or simply unsetting it.
# This ensures a local filesystem backend, as stated in the scenario.
# ----------------------------------------------------------------------
mlflow.set_tracking_uri("sqlite:///mlruns.db") # Use a local SQLite for local experiments
print(f"Tracking URI: '{mlflow.get_tracking_uri()}'")
print("This scenario uses a local SQLite database for the backend.")

mlflow.set_experiment("local-filesystem-experiment")

with mlflow.start_run():
    mlflow.set_tag("scenario", "1")
    mlflow.log_param("model", "DecisionTree")
    mlflow.log_metric("accuracy", 0.85)

# Test Model Registry access (should fail)
try:
    client = mlflow.tracking.MlflowClient()
    client.search_registered_models()
    print("Model Registry is accessible.")
except MlflowException as e:
    print(f"\nExpected Failure: It's not possible to fully access the Model Registry in this setup. Error: {e}")

Tracking URI: 'sqlite:///mlruns.db'
This scenario uses a local SQLite database for the backend.


2025/10/29 06:58:08 INFO mlflow.store.db.utils: Creating initial MLflow database tables...
2025/10/29 06:58:08 INFO mlflow.store.db.utils: Updating database tables
2025-10-29 06:58:08 INFO  [alembic.runtime.migration] Context impl SQLiteImpl.
2025-10-29 06:58:08 INFO  [alembic.runtime.migration] Will assume non-transactional DDL.
2025-10-29 06:58:08 INFO  [alembic.runtime.migration] Running upgrade  -> 451aebb31d03, add metric step
2025-10-29 06:58:08 INFO  [alembic.runtime.migration] Running upgrade 451aebb31d03 -> 90e64c465722, migrate user column to tags
2025-10-29 06:58:08 INFO  [alembic.runtime.migration] Running upgrade 90e64c465722 -> 181f10493468, allow nulls for metric values
2025-10-29 06:58:08 INFO  [alembic.runtime.migration] Running upgrade 181f10493468 -> df50e92ffc5e, Add Experiment Tags Table
2025-10-29 06:58:08 INFO  [alembic.runtime.migration] Running upgrade df50e92ffc5e -> 7ac759974ad8, Update run tags with larger limit
2025-10-29 06:58:08 INFO  [alembic.runtime.mig

Model Registry is accessible.


In [16]:
import mlflow

# ----------------------------------------------------------------------
# 💡 CRITICAL: URI set to the local server, accessible via localhost:5001.
# Assumes server was launched with: mlflow server --backend-store-uri sqlite:///backend.db --port 5001
# ----------------------------------------------------------------------
mlflow.set_tracking_uri("http://127.0.0.1:5001")
print(f"Tracking URI: '{mlflow.get_tracking_uri()}'")
print("This scenario uses a local tracking server with SQLite backend.")

mlflow.set_experiment("local-server-sqlite-experiment")

with mlflow.start_run() as run:
    mlflow.set_tag("scenario", "2")
    mlflow.log_param("model", "Ridge")
    mlflow.log_metric("loss", 0.05)
    
    # Log a dummy model to test Model Registry access
    mlflow.sklearn.log_model(
        sk_model=mlflow.sklearn.load_model(f"runs:/{run.info.run_id}/model"),
        artifact_path="models",
        registered_model_name='local-sqlite-model-test'
    )
    print("\nModel successfully logged and registered (if it didn't exist). Model Registry is working.")

2025/10/29 06:58:21 INFO mlflow.tracking.fluent: Experiment with name 'local-server-sqlite-experiment' does not exist. Creating a new experiment.


Tracking URI: 'http://127.0.0.1:5001'
This scenario uses a local tracking server with SQLite backend.


/home/ubuntu/anaconda3/envs/.venv/lib/python3.13/site-packages/tqdm/auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


🏃 View run nosy-elk-704 at: http://127.0.0.1:5001/#/experiments/754476174928006296/runs/aa6a0dce4bd240d1abde6b214344443f
🧪 View experiment at: http://127.0.0.1:5001/#/experiments/754476174928006296


MlflowException: Failed to download artifacts from path 'model', please ensure that the path is correct.

In [ ]:
import mlflow

# ----------------------------------------------------------------------
# 💡 CRITICAL: URI set to the local server, accessible via localhost:5001.
# Assumes server was launched with: mlflow server --backend-store-uri sqlite:///backend.db --port 5001
# ----------------------------------------------------------------------
mlflow.set_tracking_uri("http://127.0.0.1:5001")
print(f"Tracking URI: '{mlflow.get_tracking_uri()}'")
print("This scenario uses a local tracking server with SQLite backend.")

mlflow.set_experiment("local-server-sqlite-experiment")

with mlflow.start_run() as run:
    mlflow.set_tag("scenario", "2")
    mlflow.log_param("model", "Ridge")
    mlflow.log_metric("loss", 0.05)
    
    # Log a dummy model to test Model Registry access
    mlflow.sklearn.log_model(
        sk_model=mlflow.sklearn.load_model(f"runs:/{run.info.run_id}/model"),
        artifact_path="models",
        registered_model_name='local-sqlite-model-test'
    )
    print("\nModel successfully logged and registered (if it didn't exist). Model Registry is working.")

### Creating an experiment and logging a new run

### Interacting with the model registry